In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import tempfile
import pandas as pd
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import dataset_schema
from tensorflow_transform.tf_metadata import dataset_metadata
import apache_beam as beam
import tensorflow_transform.beam as tft_beam
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from apache_beam.options.pipeline_options import PipelineOptions
import tensorflow_transform.beam.impl as beam_impl
 
from measurements import measure

tf.__version__

/home/wgiersche/workspace/venv3/lib/python3.5/site-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


'1.13.1'

A little trick: Take a note of the temporary directory containing the data for re-use in other notebooks.

In [3]:
temp_dir = "temp_dir = '%s'" % tempfile.mkdtemp()
with open('temp_dir.py', 'w') as file:
    file.write(temp_dir)
from temp_dir import temp_dir

In [4]:
temp_dir

'/tmp/tmpxhfk07ff'

In [5]:

# Signature data: Just the way it'll arrive at prediction time
signature_csv_train = os.path.join(temp_dir, "signature_train.csv")
signature_csv_eval = os.path.join(temp_dir, "signature_eval.csv")
signature_csv_test = os.path.join(temp_dir, "signature_test.csv")

# Training data: maybe scaled or further pre-processed.
training_csv = os.path.join(temp_dir, "training.csv")
eval_csv = os.path.join(temp_dir, "eval.csv")

# TFRecord: Allows for high performance input into computational graphs
training_tfr = os.path.join(temp_dir, "training.tfr")
eval_tfr = os.path.join(temp_dir, "eval.tfr")

print("You can find the files at:")
print(signature_csv_train, training_csv, training_tfr)

You can find the files at:
/tmp/tmpxhfk07ff/signature_train.csv /tmp/tmpxhfk07ff/training.csv /tmp/tmpxhfk07ff/training.tfr


In [7]:
data = measure(5)
data.to_csv(signature_csv_train, index=None)
data = pd.read_csv(signature_csv_train)
data.head()

,beta1,beta2,hour,humidity,weekday
0,0.868526,-2.455402,19,23.589506,3
1,-0.975189,-3.637814,2,20.452662,3
2,0.137844,2.707074,2,19.448784,1
3,-4.185026,-0.508476,12,7.901505,3
4,0.491551,-4.591976,19,19.918408,5


### Specify the input and output formats

In [8]:
ORDERED_SIGNATURE_COLUMNS=["beta1", "beta2", "hour", "humidity", "weekday"]
header = bytes(",".join(ORDERED_SIGNATURE_COLUMNS), 'UTF-8')

In [9]:
feature_spec = {
    'beta1': tf.io.FixedLenFeature([1], tf.float32),
    'beta2': tf.io.FixedLenFeature([1], tf.float32),
    'weekday': tf.io.FixedLenFeature([1], tf.int64),
    'hour': tf.io.FixedLenFeature([1], tf.int64),
    'humidity': tf.io.FixedLenFeature([1], tf.float32)
}
schema = dataset_schema.from_feature_spec(feature_spec)

### Create an encoder and test it

In [10]:
csv_encoder = tft.coders.CsvCoder(ORDERED_SIGNATURE_COLUMNS, schema)
records = csv_encoder.decode("10.201, 10.101, 3,1.234,4")
print(records)
csv_encoder.encode(records)

{'humidity': array([1.234], dtype=float32), 'beta1': array([10.201], dtype=float32), 'hour': array([3]), 'weekday': array([4]), 'beta2': array([10.101], dtype=float32)}


b'10.201,10.101,3,1.234,4'

### The Apache Beam pipeline 

In [11]:
def process_data(row):
    print(row)
    return row

### Dry run - Everything working?

In [13]:
with beam.Pipeline('DirectRunner', PipelineOptions()) as p:

    csv_encoder = tft.coders.CsvCoder(ORDERED_SIGNATURE_COLUMNS, schema)    

    _ = (p 
         | 'read_from_csv' >> beam.io.ReadFromText(
             file_pattern=signature_csv_train, coder=csv_encoder, skip_header_lines=1)
         
         | 'process_records' >> beam.Map(process_data)
         
         | 'write_to_csv' >> beam.io.WriteToText(
             file_path_prefix=training_csv, coder=csv_encoder, header=header)
        )


{'humidity': array([23.589506], dtype=float32), 'beta1': array([0.8685255], dtype=float32), 'hour': array([19]), 'weekday': array([3]), 'beta2': array([-2.4554024], dtype=float32)}
{'humidity': array([20.452662], dtype=float32), 'beta1': array([-0.97518927], dtype=float32), 'hour': array([2]), 'weekday': array([3]), 'beta2': array([-3.6378136], dtype=float32)}
{'humidity': array([19.448784], dtype=float32), 'beta1': array([0.13784397], dtype=float32), 'hour': array([2]), 'weekday': array([1]), 'beta2': array([2.7070744], dtype=float32)}
{'humidity': array([7.9015045], dtype=float32), 'beta1': array([-4.1850257], dtype=float32), 'hour': array([12]), 'weekday': array([3]), 'beta2': array([-0.5084763], dtype=float32)}
{'humidity': array([19.918407], dtype=float32), 'beta1': array([0.49155068], dtype=float32), 'hour': array([19]), 'weekday': array([5]), 'beta2': array([-4.5919757], dtype=float32)}


In [14]:
!echo "Reading from: " $training_csv*
!cat $training_csv*

Reading from:  /tmp/tmpxhfk07ff/training.csv-00000-of-00001
beta1,beta2,hour,humidity,weekday
0.8685255,-2.4554024,19,23.589506,3
-0.97518927,-3.6378136,2,20.452662,3
0.13784397,2.7070744,2,19.448784,1
-4.1850257,-0.5084763,12,7.9015045,3
0.49155068,-4.5919757,19,19.918407,5


### Serious transformation: Scale $\beta_1$ and $\beta_2$

In [15]:
for phase_csv in [signature_csv_train, signature_csv_eval, signature_csv_test]:
    measure(20000).to_csv(phase_csv, index=None)

In [16]:
def process_data(row):
    for c in ['beta1', 'beta2']:
        row[c] = tft.scale_to_0_1(row[c])
    return row

In [17]:
signature_metadata = dataset_metadata.DatasetMetadata(schema)

In [18]:
csv_encoder = tft.coders.CsvCoder(ORDERED_SIGNATURE_COLUMNS, schema)    
tfr_encoder = tft.coders.ExampleProtoCoder(schema)            

metadata_dir = os.path.join(temp_dir, "metadata")
with beam.Pipeline('DirectRunner', PipelineOptions()) as p:

    #
    # The context is provided for the AnalyseAndTransform step.
    # That step needs a hand to do its magic.
    #
    with tft_beam.Context(temp_dir=temp_dir):

        #
        # Read from csv, skip headers. Note that we use ordered columns in the encoder
        # 
        signature_data = ( p | 'read_from_csv' 
            >> beam.io.ReadFromText(
                 file_pattern=signature_csv_train, 
                coder=csv_encoder, skip_header_lines=1))

        #
        # attach the metadata: required for AnalyzeAndTransform
        #
        signature_data = ( signature_data, signature_metadata)

        #
        # Do the magic two steps and return also the transform-function
        #
        data_and_metadata, transform_fn = ( signature_data | "AnalyzeAndTransform" 
                         >> beam_impl.AnalyzeAndTransformDataset(process_data))
        
        #
        # split data and metadata
        #
        training_data, training_metadata = data_and_metadata

        #
        # Write the resulting data to a csv file
        #
        _ = (training_data | 'write_to_csv' 
             >> beam.io.WriteToText(
                 file_path_prefix=training_csv, coder=csv_encoder, header=header))

        #
        # For production purposes, we use the TFRecord format
        #
        _ = (training_data | 'write_to_tfr' 
             >> beam.io.WriteToTFRecord(
                 file_path_prefix=training_tfr, coder=tfr_encoder))

        
        #  Process evaluation data with the obtained transform_fn
        #
        signature_data = ( p | 'read_from_csv_eval' 
            >> beam.io.ReadFromText(
                 file_pattern=signature_csv_eval, coder=csv_encoder, skip_header_lines=1))

        signature_data = (signature_data, signature_metadata)

        # Use the transform_fn of the previous step here
        eval_data, _ = ((signature_data, transform_fn) 
                     | "TransformEval" >> tft_beam.TransformDataset())

        _ = (eval_data | 'write_to_tfr_eval' 
             >> beam.io.WriteToTFRecord(
                 file_path_prefix=eval_tfr, coder=tfr_encoder))

        
        
        #
        # Eventually, save the transform function for re-use at prediction time.
        #
        _ = (transform_fn | 'WriteTransformFn' 
             >> transform_fn_io.WriteTransformFn(metadata_dir))


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


INFO:tensorflow:Assets added to graph.


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [19]:
!echo "Reading from: " $training_csv*
!echo
!cat $training_csv* | tail -5
!echo
!echo metadata is here:
!ls $metadata_dir
!echo 
!echo "TFRecords are here: " $training_tfr*

Reading from:  /tmp/tmpxhfk07ff/training.csv-00000-of-00001

0.6771585,0.69741875,3,19.253529,3
0.9618148,0.46862492,3,26.659876,2
0.8357462,0.7557871,11,22.597065,5
0.3334872,0.17532052,9,20.044199,3
0.6912456,0.27015296,14,32.35936,5

metadata is here:
transformed_metadata  transform_fn

TFRecords are here:  /tmp/tmpxhfk07ff/training.tfr-00000-of-00001


### Wrap up

We're now able to analyze and process any size of data, scale particular features to the interval $[0, 1]$ while saving the function that actually did it for later. We'll need that function to apply exactly the same scaling to the incoming data at prediction time. 
Note that by simply swapping ```'DirectRunner'``` in the Apache Beam pipeline by ```'DataFlowRunner'``` in an adequately configured GCP environment, we could have the pipeline executed on an arbitrarily large cluster.